## DistillBert

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="typeform/distilbert-base-uncased-mnli")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/258 [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


### Zero Shot Prompting

In [ ]:
from transformers import pipeline

review = "The movie had great special effects but the plot was very weak and predictable."

candidate_labels = ["Positive", "Negative", "Neutral"]

result = classifier(review, candidate_labels)

sentiment_label = result['labels'][result['scores'].index(max(result['scores']))]

print("Zero-shot Sentiment:", sentiment_label)


Zero-shot Sentiment: Negative


### One Shot Prompting

In [ ]:
review = "The movie had great special effects but the plot was very weak and predictable."

prompt = """Classify the sentiment of the following movie reviews as Positive, Negative, or Neutral:

Review: "The movie was a fantastic blend of action and emotion, loved it!"
Sentiment: Positive

Review: "{}"
Sentiment:""".format(review)


result = classifier(prompt, candidate_labels)

sentiment_label = result['labels'][result['scores'].index(max(result['scores']))]

print("One-shot Sentiment:", sentiment_label)


One-shot Sentiment: Positive


### Few Shot Prompting

In [ ]:
review = "The movie had great special effects but the plot was very weak and predictable."

prompt = """Classify the sentiment of the following movie reviews as Positive, Negative, or Neutral:

Review: "The movie was a fantastic blend of action and emotion, loved it!"
Sentiment: Positive

Review: "The film was too slow, and I lost interest halfway through."
Sentiment: Negative

Review: "It was a decent watch, not bad but not amazing either."
Sentiment: Neutral

Review: "{}"
Sentiment:""".format(review)


result = classifier(prompt, candidate_labels)
sentiment_label = result['labels'][result['scores'].index(max(result['scores']))]

# Print the classification result
print("Few-shot Sentiment:", sentiment_label)



Few-shot Sentiment: Neutral


## Tiny Llama

### Loading the model from Hugging Face Pipeline

In [ ]:
# Install transformers from source - only needed for versions <= v4.34
# pip install git+https://github.com/huggingface/transformers.git
# pip install accelerate

import torch
from transformers import pipeline

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])



config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>
There is no exact scientifically proven answer to this question. However, some estimates suggest that humans can eat around 150-200 grams of food per hour. This is based on the fact that humans have an appetite that can sustain them for around 6-8 hours, and eating roughly 100-150 grams of food per hour would be enough to satisfy this appetite. However, the exact amount of food that one can consume in a single sitting may vary depending on factors such as body size, age, and physical activity levels. It's always best to consult a healthcare professional before consuming a large amount of food in one sitting.


### Zero Shot Prompting

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that classifies the sentiment of movie reviews as Positive, Negative, or Neutral.",
    },
    {
        "role": "user",
        "content": "Classify the sentiment of the following movie review: 'The movie had great special effects but the plot was very weak and predictable.'"
    },
]

# Applying the chat template and generating a response
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])


<|system|>
You are a helpful assistant that classifies the sentiment of movie reviews as Positive, Negative, or Neutral.</s>
<|user|>
Classify the sentiment of the following movie review: 'The movie had great special effects but the plot was very weak and predictable.'</s>
<|assistant|>
The sentiment of the movie review "The movie had great special effects but the plot was very weak and predictable" is Positive.


### One Shot Prompting

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that classifies the sentiment of movie reviews as Positive, Negative, or Neutral.",
    },
    {
        "role": "user",
        "content": """Classify the sentiment of the following movie reviews:

Review: "The movie was a fantastic blend of action and emotion, loved it!"
Sentiment: Positive

Review: "The movie had great special effects but the plot was very weak and predictable."
Sentiment:"""
    },
]

# Applying the chat template and generating a response
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])


<|system|>
You are a helpful assistant that classifies the sentiment of movie reviews as Positive, Negative, or Neutral.</s>
<|user|>
Classify the sentiment of the following movie reviews:

Review: "The movie was a fantastic blend of action and emotion, loved it!"
Sentiment: Positive

Review: "The movie had great special effects but the plot was very weak and predictable."
Sentiment:</s>
<|assistant|>
Sentiment: Negative

Review: "The movie was not very engaging and lacked any sense of character development or storyline."
Sentiment:

Review: "The movie was an absolute mess, with terrible pacing and lackluster performances."
Sentiment:

Review: "The movie had a few great moments but was overstuffed with unnecessary plot twists and characters."
Sentiment:


### Few Shot Prompting

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that classifies the sentiment of movie reviews as Positive, Negative, or Neutral.",
    },
    {
        "role": "user",
        "content": """Classify the sentiment of the following movie reviews:

Review: "The movie was a fantastic blend of action and emotion, loved it!"
Sentiment: Positive

Review: "The film was too slow, and I lost interest halfway through."
Sentiment: Negative

Review: "It was a decent watch, not bad but not amazing either."
Sentiment: Neutral

Review: "The movie had great special effects but the plot was very weak and predictable."
Sentiment:"""
    },
]

# Applying the chat template and generating a response
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])


<|system|>
You are a helpful assistant that classifies the sentiment of movie reviews as Positive, Negative, or Neutral.</s>
<|user|>
Classify the sentiment of the following movie reviews:

Review: "The movie was a fantastic blend of action and emotion, loved it!"
Sentiment: Positive

Review: "The film was too slow, and I lost interest halfway through."
Sentiment: Negative

Review: "It was a decent watch, not bad but not amazing either."
Sentiment: Neutral

Review: "The movie had great special effects but the plot was very weak and predictable."
Sentiment:</s>
<|assistant|>
Based on the provided movie reviews, the sentiment of each review can be classified as Positive, Negative, or Neutral based on the following criteria:

1. Review 1: "The movie was a fantastic blend of action and emotion, loved it!" - Sentiment: Positive
2. Review 2: "The film was too slow, and I lost interest halfway through." - Sentiment: Negative
3. Review 3: "It was a decent watch, not bad but not amazing either.

## GPT-2

### Loading the model from Hugging Face Pipeline

In [ ]:
from transformers import pipeline, set_seed

# Initialize the GPT-2 pipeline for text generation
generator = pipeline('text-generation', model='gpt2')
set_seed(42)


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


### Zero Shot Prompting

In [ ]:
review = "The movie had great special effects but the plot was very weak and predictable."

# Zero-shot prompt
prompt = f"Classify the sentiment of the following movie review as Positive, Negative, or Neutral:\n\nReview: '{review}'\nSentiment:"

# Generate response
response = generator(prompt, max_length=100, num_return_sequences=1)

# Print the generated response
print("Zero-shot Sentiment:", response[0]['generated_text'])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Zero-shot Sentiment: Classify the sentiment of the following movie review as Positive, Negative, or Neutral:

Review: 'The movie had great special effects but the plot was very weak and predictable.'
Sentiment: 'The movie has good action but the characters were not very likable as compared to the other movies i reviewed.'

Positive: 'Pretty disappointing but the story was strong.'

Negative: 'Just a boring little joke.'

So that's it! I hope you


### One Shot Prompting

In [ ]:
review = "The movie had great special effects but the plot was very weak and predictable."

# One-shot prompt with an example
prompt = """Classify the sentiment of the following movie reviews as Positive, Negative, or Neutral:

Review: "The movie was a fantastic blend of action and emotion, loved it!"
Sentiment: Positive

Review: "{}"
Sentiment:""".format(review)

# Generate response
response = generator(prompt, max_length=100, num_return_sequences=1)

# Print the generated response
print("One-shot Sentiment:", response[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


One-shot Sentiment: Classify the sentiment of the following movie reviews as Positive, Negative, or Neutral:

Review: "The movie was a fantastic blend of action and emotion, loved it!"
Sentiment: Positive

Review: "The movie had great special effects but the plot was very weak and predictable."
Sentiment: Negative

Review: "Not as good as the previous review, but I felt very satisfied. The plot was not bad!"

Reaction: Negative

Review


### Few Shot Prompting

In [ ]:
review = "The movie had great special effects but the plot was very weak and predictable."

# Few-shot prompt with multiple examples
prompt = """Classify the sentiment of the following movie reviews as Positive, Negative, or Neutral:

Review: "The movie was a fantastic blend of action and emotion, loved it!"
Sentiment: Positive

Review: "The film was too slow, and I lost interest halfway through."
Sentiment: Negative

Review: "It was a decent watch, not bad but not amazing either."
Sentiment: Neutral

Review: "{}"
Sentiment:""".format(review)

# Generate response
response = generator(prompt, max_length=120, num_return_sequences=1)

# Print the generated response
print("Few-shot Sentiment:", response[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Few-shot Sentiment: Classify the sentiment of the following movie reviews as Positive, Negative, or Neutral:

Review: "The movie was a fantastic blend of action and emotion, loved it!"
Sentiment: Positive

Review: "The film was too slow, and I lost interest halfway through."
Sentiment: Negative

Review: "It was a decent watch, not bad but not amazing either."
Sentiment: Neutral

Review: "The movie had great special effects but the plot was very weak and predictable."
Sentiment: Positive

Review: "The movie was
